In [6]:

from AlgorithmImports import *
# Quick end-to-end test for V-bar footprint aggregation and storage
from datetime import date
import pandas as pd

from orchestrator import run
from footprint_storage import get_year_file_path

# Assumes `qb` is available in the Research environment
qb = QuantBook()


symbol = qb.add_future(Futures.Indices.NASDAQ_100_E_MINI, Resolution.SECOND).symbol

start = date(2024, 11, 1)
end = date(2024, 11, 10)

v_unit = 500  # minimal volume unit per bar
# Set tick_size per instrument; examples: NQ=0.25, GC=0.1

sec = qb.Securities[symbol]
tick_size = sec.SymbolProperties.MinimumPriceVariation


run(qb=qb, symbol=symbol, start_date=start, end_date=end, v_unit=v_unit, tick_size=tick_size, force_recompute=False)

print("Done.")


Done.


In [ ]:
# Inspect written parquet and preview a day's bars
import pyarrow.parquet as pq
import os

from footprint_storage import get_year_file_path

year = start.year
p = get_year_file_path(symbol, year)
print("Year file:", p, "Exists:", os.path.exists(p))

if os.path.exists(p):
    pf = pq.ParquetFile(p)
    print("Row groups (days):", pf.num_row_groups)
    # Read full file as pandas for quick look
    df_year = pd.read_parquet(p, engine="pyarrow")
    print(df_year.head())
    print("Unique trade_date count:", df_year["trade_date"].nunique())



In [ ]:
# 从年度文件读取指定交易日，重建 FootprintBar 列表并绘图（复用 test.ipynb 逻辑）
from footprint_storage import read_day_as_footprint_bars, get_metadata_path
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
from collections import defaultdict
from footprint_utils import price_to_bucket

# 选择绘图日期（YYYYMMDD）
target_date_int = int(start.strftime("%Y%m%d"))  # 可改为其他交易日
# 从元数据取 tick_size（也可直接传入 read_day_as_footprint_bars 的 tick_size）
with open(get_metadata_path(symbol, start.year), "r", encoding="utf-8") as f:
    _meta = json.load(f)
meta_tick_size = float(_meta["tick_size"])

footprint_bars = read_day_as_footprint_bars(symbol, start.year, target_date_int)
print("bars:", len(footprint_bars))
print(footprint_bars[0])
# 复用 test.ipynb 的绘图（略微变量名对齐）
group_tick_factor = 4
merged_tick_size = meta_tick_size * group_tick_factor

footprint_subset = footprint_bars[:20]
fig, ax = plt.subplots(figsize=(40, 20))

red_cmap = LinearSegmentedColormap.from_list("custom_red", ["#FFCDD2", "#B71C1C"])
green_cmap = LinearSegmentedColormap.from_list("custom_green", ["#C8E6C9", "#1B5E20"])

bar_width = 0.8
bar_padding = 0.2

# 重新计算最大成交量（按合并后的价格桶）
all_aggregated_volumes = []
for fp_bar in footprint_subset:
    aggregated_volume_at_price = defaultdict(lambda: {"bid": 0.0, "ask": 0.0})
    for price_bucket, volumes in fp_bar.volume_at_price.items():
        merged_bucket_price = price_to_bucket(price_bucket, merged_tick_size)
        aggregated_volume_at_price[merged_bucket_price]["bid"] += volumes["bid"]
        aggregated_volume_at_price[merged_bucket_price]["ask"] += volumes["ask"]
    all_aggregated_volumes.extend([v["bid"] for v in aggregated_volume_at_price.values()])
    all_aggregated_volumes.extend([v["ask"] for v in aggregated_volume_at_price.values()])
max_vol = max(all_aggregated_volumes) if all_aggregated_volumes else 1

# 绘制
for i, fp_bar in enumerate(footprint_subset):
    x_pos = i * (bar_width + bar_padding)
    if fp_bar.close >= fp_bar.open:
        rect_color = 'green'
        body_bottom = fp_bar.open
        body_height = fp_bar.close - fp_bar.open
    else:
        rect_color = 'red'
        body_bottom = fp_bar.close
        body_height = fp_bar.open - fp_bar.close
    if body_height == 0:
        body_height = meta_tick_size * 0.1
    rect = patches.Rectangle((x_pos - 0.05, body_bottom), 0.1, body_height,
                             linewidth=1, edgecolor=rect_color, facecolor=rect_color, alpha=0.3)
    ax.add_patch(rect)
    ax.plot([x_pos, x_pos], [fp_bar.low, fp_bar.high], color=rect_color, linewidth=1, alpha=0.5)

    aggregated_volume_at_price = defaultdict(lambda: {"bid": 0.0, "ask": 0.0})
    for price_bucket, volumes in fp_bar.volume_at_price.items():
        merged_bucket_price = price_to_bucket(price_bucket, merged_tick_size)
        aggregated_volume_at_price[merged_bucket_price]["bid"] += volumes["bid"]
        aggregated_volume_at_price[merged_bucket_price]["ask"] += volumes["ask"]
    sorted_aggregated_prices = sorted(aggregated_volume_at_price.keys())

    total_delta = fp_bar.delta
    for price in sorted_aggregated_prices:
        volumes = aggregated_volume_at_price[price]
        sell_vol_str = f"{int(volumes['bid'])}"
        sell_color = red_cmap(volumes['bid'] / max_vol if max_vol > 0 else 0)
        ax.text(x_pos - bar_width/2, price, sell_vol_str, ha='right', va='center', fontsize=8,
                color='white', backgroundcolor=sell_color)

        buy_vol_str = f"{int(volumes['ask'])}"
        buy_color = green_cmap(volumes['ask'] / max_vol if max_vol > 0 else 0)
        ax.text(x_pos + bar_width/2, price, buy_vol_str, ha='left', va='center', fontsize=8,
                color='white', backgroundcolor=buy_color)

    ax.text(x_pos, fp_bar.high, f"Δ={int(total_delta)}", ha='center', va='bottom', fontsize=9, color='blue')

title_symbol = symbol.value if hasattr(symbol, "value") else str(symbol)
ax.set_title(f"Footprint Cluster Chart for {title_symbol} on {target_date_int}")
ax.set_xlabel("Bars (V-bars)")
ax.set_ylabel("Price")
ax.set_xticks([i * (bar_width + bar_padding) for i in range(len(footprint_subset))])
ax.set_xticklabels([fp.time.strftime('%H:%M:%S') for fp in footprint_subset], rotation=45)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# Debug：检查第一根和前几根 V-bar 的 footprint 是否为空，以及聚合后层数与体量
from collections import defaultdict
from footprint_utils import price_to_bucket

assert len(footprint_bars) > 0, "没有读取到任何 V-bar"
print("总Vbars:", len(footprint_bars))

def inspect_bar(fp_bar, merged_tick_size, topn=10):
    print("start:", fp_bar.time, "end:", fp_bar.end_time)
    print("OHLC:", fp_bar.open, fp_bar.high, fp_bar.low, fp_bar.close)
    print("total/buy/sell:", fp_bar.total_volume, fp_bar.buy_volume, fp_bar.sell_volume, "delta:", fp_bar.delta)
    print("原始价位层数:", len(fp_bar.volume_at_price))

    # 打印前N个价位样例
    if fp_bar.volume_at_price:
        items = sorted(fp_bar.volume_at_price.items())[:topn]
        print("样例[price -> bid, ask]:")
        for p, v in items:
            print(f"{p:.2f} -> {int(v['bid'])}, {int(v['ask'])}")
    else:
        print("volume_at_price 为空")

    # 合并后
    agg = defaultdict(lambda: {"bid": 0, "ask": 0})
    for price_bucket, v in fp_bar.volume_at_price.items():
        merged_price = price_to_bucket(price_bucket, merged_tick_size)
        agg[merged_price]["bid"] += int(v["bid"])
        agg[merged_price]["ask"] += int(v["ask"])

    print("合并后层数:", len(agg))
    if agg:
        vals = list(agg.items())
        vals.sort()
        print("合并后样例:")
        for p, v in vals[:topn]:
            print(f"{p:.2f} -> {v['bid']}, {v['ask']}")
        print("合并后总量校验: bid+ask =", sum(v["bid"]+v["ask"] for v in agg.values()))
    print("-" * 60)

merged_tick_size = meta_tick_size * 4  # 与你绘图一致
# 检查前3根
for i in range(min(3, len(footprint_bars))):
    inspect_bar(footprint_bars[i], merged_tick_size)